In [ ]:
import numpy as np
import cv2

In [ ]:
# tune parameters
down_left_margin = 240
down_right_margin = 130
up_left_margin = 570
up_right_margin = 520
sight_range = 500
red_zone = (300, 1000)
dist_thresold = 80

LeftLine =  np.zeros((1, 4))
RightLine =  np.zeros((1, 4))

In [ ]:
def make_coord(img, line_parameters):
    ''' the function create coordinates from the slope and the intercept
    '''
    slope, intercept = line_parameters
    # y1 is equal to image height as we want to start detecting from the bottm of the photo
    y1 = img.shape[0]
    # set the height of detected lines to 4/5 of the photo
    y2 = int(img.shape[0] * 0.75)
    
    # calculate x coordinates
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)
    
    return np.array([x1, y1, x2, y2])    
    
def average_slope_intercept(img, lines):
    ''' the function calculate the average slope & intercept
    of detecte lines
    '''
    # create empty lists for the slope & intercept of left lines
    lt_slope = []
    lt_coord = []
    
    # create empty lists for the slope & intercept of right lines
    rt_slope = []
    rt_coord = []
    
    # loop over detected lines
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        # polyfit returns an eq that goes through all the points 
        #that minimise th squared error
        # it returns [slope, y-intercept]
        # 1 means degree 1
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        
        # ignore lines with sharp angles
        if abs(parameters[0]) > 0.6:
            # classify the slopes into right & left
            if parameters[0] < 0:
                lt_slope.append(parameters[0])
                lt_coord.append(make_coord(img, parameters))
            else:
                rt_slope.append(parameters[0])
                rt_coord.append(make_coord(img, parameters))

    return np.array([list(zip(lt_slope, lt_coord)), list(zip(rt_slope, rt_coord))])

def canny(img):
    ''' function creates canny image
    '''
    # convert to gray scale
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # apply gaussian blur
    img = cv2.GaussianBlur(img, (5, 5), 0)
    # edge detection
    canny = cv2.Canny(img, 20,80)
    return canny

def region_of_interest(img):
    '''The function defines the camera vision area,
    parameters can be tuned to be adjusted with the position of the camera
    '''
    # get image shape
    h = img.shape[0]
    w = img.shape[1]
    
    # define the scope of the camera
    ploygons = np.array([[(down_left_margin, h), (w - down_right_margin, h), 
                          (w-up_right_margin+10, sight_range), (up_left_margin, sight_range)]])
    
    # define a blank mask in the size of the image
    mask = np.zeros_like(img)
    
    # fill the scope of the camera in the mask with white
    cv2.fillPoly(mask, ploygons, 255)
    
    # mask the image
    masked_img = cv2.bitwise_and(img, mask)
    return masked_img

def display_lines(img, lines):
    ''' the function shows lines on the screen'''
    
    line_img = np.zeros_like(img)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.astype(int).reshape(4,)
            cv2.line(line_img, (x1,y1), (x2, y2), (255, 0, 0), 4)
            
    return line_img

def find_bundles(img, lines):
    '''
    each line on the road is detected many times, 
    which make a bundle of lines for the same line.
    The function finds these bundles, and make one line of each bundle 
    by returning the average
    '''
    # create empty lists
    final_vectors = []
    bundle_vectors = []
 
    for s, coord in lines:
        # find lines with almost the same slope
        slope_diff = [lines[i][1] for i in range(len(lines)) if abs(s - lines[i][0]) < 0.2]
        # find lines within the distance threshold, this parameter can be tuned
        x1_diff = [slope_diff[i] for i in range(len(slope_diff)) if abs(coord[0] - slope_diff[i][0]) < dist_thresold]
        
        # check if the line is not already in the bundle list
        if any(np.array_equal(x1_diff, x) for x in bundle_vectors) == False:
            bundle_vectors.append(x1_diff) 
        
    # take the average of the bundles
    for v in bundle_vectors:
        if len(v) > 1:
            final_vectors.append(np.average(v, axis=0))

    return final_vectors

In [ ]:
def FindLines(frame):
    global LeftLine, RightLine
    
    h = frame.shape[0]
        
    # get a canny of the frame
    canny_img = canny(frame)
    # mask the scope of the camera area
    cropped_img = region_of_interest(canny_img)
    # detect line in the frame
    lines = cv2.HoughLinesP(cropped_img, 2, np.pi/180, 80, np.array([]), minLineLength=8, maxLineGap= 10)

    # get the slope & intercept of the detected lines
    lt_lines, rt_lines = average_slope_intercept(frame, lines)

    # clean detected lines by removing lines outside the image
    r_lines = [x for x in rt_lines if abs(x[1][0]) < frame.shape[1]]
    r_lines = [x for x in rt_lines if abs(x[1][2]) < frame.shape[1]]
    l_lines = [x for x in lt_lines if abs(x[1][0]) < frame.shape[1]]
    l_lines = [x for x in lt_lines if abs(x[1][2]) < frame.shape[1]]

    # remove lines in the red zone, area outside the scope of the camera 
    r_lines = [x for x in rt_lines if (abs(x[1][0]) < red_zone[0]) or (abs(x[1][0]) > red_zone[1])]
    r_lines = [x for x in rt_lines if abs(x[1][2] < red_zone[0]) or (abs(x[1][0]) > red_zone[1])]
    l_lines = [x for x in lt_lines if (abs(x[1][0]) < red_zone[0]) or (abs(x[1][0]) > red_zone[1])]

    # get the average of the same detected lines in the left
    lt_lines = find_bundles(frame, l_lines)
    # if no line is detected, assign to it last known value
    if lt_lines == []:
        lt_lines = LeftLine
    # # update last known value of line 
    else:
        LeftLine =  lt_lines  

    # get the average of the same dected lines in the right    
    rt_lines = find_bundles(frame, r_lines)
    # if no line is detected, assign to it last known value
    if rt_lines == []:
        rt_lines = RightLine
    # update last known value of line 
    else:
        RightLine = rt_lines

    # combine the detected left & right lines together
    lines = np.concatenate((lt_lines, rt_lines), axis=0)

    line_img = display_lines(frame, lines)
    final_img = cv2.addWeighted(frame, 0.9, line_img, 1, 1)
    
    return final_img

## Calibrate the parameters

In [ ]:
cap = cv2.VideoCapture('challenge_video.mp4')
ret, frame = cap.read()


cv2.imshow("Frame", FindLines(frame))
cv2.waitKey(0)
cv2.destroyAllWindows()

## Detect lines

In [ ]:
# read the video
cap = cv2.VideoCapture('challenge_video.mp4') #('test2.mp4')

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        final_img = FindLines(frame)
        cv2.imshow("Frame", final_img) 
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break
        
cap.release()
print('finished')
cv2.destroyAllWindows()